Collection of functions needed to create and interact with the model. This refers to model creation, interpolation, interaction (add points etc.) and analysis. It does not however refer to visualizations as this will be done in the frondend.

In [40]:
import gempy as gp
import numpy as np
import pandas as pd

## InputData

In [2]:
# init model
geo_model = gp.create_model('GeoModel')

In [5]:
# Set init data
gp.init_data(
    geo_model,
    [0,2000.,0,2000.,0,2000.],
    [50,50,50], 
    path_o = "./input_data/simple_fault_model_orientations.csv",
    path_i = "./input_data/simple_fault_model_points.csv",
    default_values=True
);

Active grids: ['regular']


In [6]:
# Map series to surfaces
gp.map_series_to_surfaces(
    geo_model,       
    {
        "Fault_Series":'Main_Fault', 
        "Strat_Series": (
            'Sandstone_2',
            'Siltstone',
            'Shale',
            'Sandstone_1',
            'basement'
        )
    },
    remove_unused_series=True
);

In [10]:
# set faults
geo_model.set_is_fault(['Fault_Series']);

Fault colors changed. If you do not like this behavior, set change_color to False.


## InterpolatorData

In [9]:
gp.set_interpolation_data(
    geo_model,
    compile_theano=True,
    theano_optimizer='fast_compile',
    verbose=[]
);

Compiling theano function...
Compilation Done!


## Compute solution

In [12]:
sol = gp.compute_model(
    geo_model,
    compute_mesh=True,
    sort_surfaces=False
);

# Data

## Series
Object that contains the properties associated with each independent scalar field.

In [21]:
faults = gp.Faults()
series = gp.Series(faults)

In [23]:
series.set_series_index(['foo', 'foo2', 'foo5', 'foo7'])
series.add_series('foo3')
series.delete_series('foo3')
series.rename_series({'foo':'boo'})

,order_series,BottomRelation
boo,1,Erosion
foo2,2,Erosion
foo5,3,Erosion
foo7,4,Erosion


## Faults
Charectirize which mathematical series behave as fault and if mentioned faults are finite or infinite. 

In [24]:
faults.set_is_fault(['boo'])

,isFault,isFinite
boo,True,False
foo2,False,False
foo5,False,False
foo7,False,False


In [26]:
faults.set_fault_relation()

,boo,foo2,foo5,foo7
boo,False,False,False,False
foo2,False,False,False,False
foo5,False,False,False,False
foo7,False,False,False,False


## Surfaces
- id refers to the order of the surfaces on the sequential pile
- values on the other hand is the final value that each voxel will have after discretization.
- needs to have an associate series object.

In [33]:
surfaces = gp.Surfaces(series)
surfaces.set_surfaces_names(['foo', 'foo2', 'foo5'])
surfaces.add_surface(['feeeee'])
surfaces.set_surfaces_values([2,2,2,5])
surfaces.add_surfaces_values([[2,2,2,6], [2,2,1,8]], ['val_foo', 'val2_foo'])
surfaces.delete_surface_values(['val_foo', 'value_0'])
surfaces.set_basement()  # required
surfaces.set_surfaces_values([[2,2,2,6], [2,2,1,8]], ['val_foo', 'val2_foo'])

,surface,series,order_surfaces,isBasement,color,id,val_foo,val2_foo
0,foo,foo7,1,False,#015482,1,2,2
1,foo2,foo7,2,False,#9f0052,2,2,2
2,foo5,foo7,3,False,#ffbe00,3,2,1
3,feeeee,foo7,4,True,#728f02,4,6,8


In [34]:
surfaces.map_series({"foo7":'foo', "boo": ('foo2','foo5', 'fee')})

,surface,series,order_surfaces,isBasement,color,id,val_foo,val2_foo
1,foo2,boo,1,False,#9f0052,1,2,2
2,foo5,boo,2,False,#ffbe00,2,2,1
0,foo,foo7,1,False,#015482,3,2,2
3,feeeee,foo7,2,True,#728f02,4,6,8


## Data

### surface points

In [42]:
surface_points = gp.SurfacePoints(surfaces)

In [44]:
surface_points.set_surface_points(pd.DataFrame(np.random.rand(6,3)), ['foo', 'foo5', 'lala', 'foo5', 'lala', 'feeeee'])

AssertionError: Some of the surface passed does not exist in the Formationobject. 2    NaN
4    NaN
Name: surface, dtype: category
Categories (4, object): [foo2, foo5, foo, feeeee]